<a href="https://colab.research.google.com/github/yezouagh/covid-19-projects/blob/master/covid_19_face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 face mask detector training script with Keras and TensorFlow

To accomplish this task, we’ll be fine-tuning the **MobileNet V2 architecture**, a highly efficient architecture that can be applied to embedded devices with limited computational capacity (ex., Raspberry Pi, Google Coral, NVIDIA Jetson Nano, etc.).

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download Face-Mask-Detector DataSet From Kaggle

When you Create your API key in Kaggle it will be automatically downloaded then upload it

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection -p /content/drive/My\ Drive/Colab\ Notebooks/face-masks-dataset/

### Unzip The DataSet

In [ ]:
!unzip -q /content/drive/My\ Drive/Colab\ Notebooks/face-masks-dataset/face-mask-detection.zip -d /content/drive/My\ Drive/Colab\ Notebooks/face-masks-dataset/

### List All files

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

# running this will list all files under the input directory
direc = '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/'
import os
for dirname, _, filenames in os.walk(direc+'images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import lxml.etree as et

for file in os.listdir(direc+"annotations"):
    xml = et.parse(direc+"annotations/"+file)
    pretty = et.tostring(xml, encoding="unicode", pretty_print=True)
    print('--------------------------------------------------------------------------------------------')
    print(pretty)

## Loading and Preprocessing Trainning Data

At this point, we’re ready to load and pre-process our training data

###Method to detect face #1

In [58]:

def detectFaceOpenCVDnn(net, frame):
    conf_threshold = 0.8
    found = False
    frameOpencvDnn = frame.copy()
    (frameHeight,frameWidth) = frameOpencvDnn.shape[:2]
    # construct a blob from the image
    # applying mean subtraction of values (104, 177, 123) for each blue, green and red channels correspondingly.
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], False, False)
    
    # pass the blob through the network and obtain the face detections
    #print("[INFO] computing face detections...")
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold and  not found:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            #if (x2 - x1) > 40 :
            #  found = True
            frameOpencvDnn = frame[y1:y2, x1:x2]
            
            cv2_imshow(frameOpencvDnn)
    
    return frameOpencvDnn


###Method to detect face #2

In [2]:
import dlib
detector = dlib.get_frontal_face_detector()
def detectFace(frame):
    frameGray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    found = False
    #(frameHeight,frameWidth) = frameGray.shape[:2]
    faces = detector(frameGray, 0)
    if len(faces) > 0:
        for face in faces:
          x1 = face.left()
          y1 = face.top()
          x2 = face.right()
          y2 = face.bottom()
          if (x2 - x1) > 40 and not found:
            found = True
            frame = frame[y1:y2, x1:x2]
          #display(x1,y1,x2,y2,frame2)
        #cv2_imshow(frame2)
    
    return frame

### Loading

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import numpy as np
import argparse
import glob
import os
import cv2
import random as rand
from imutils import paths
from google.colab.patches import cv2_imshow

direc = '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/'
face = direc+"face_detector"
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
modelFile = os.path.sep.join([face, "res10_300x300_ssd_iter_140000_fp16.caffemodel"])
configFile = os.path.sep.join([face, "deploy.prototxt"])
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
# extract a single face from a given photograph
def extract_faces_labels(filename, required_size=(224, 224)):
    labels = []
    imgs = []
    xml = et.parse(filename)
    root = xml.getroot()
    img = root[1].text
    h, w = root[2][0].text, root[2][1].text
    img = cv2.imread(direc+"images/"+img)
    img = cv2.resize(img, (int(h),int(w)))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    for i in range(4, len(root)):
        labels.append(root[i][0].text)
        info = [int(each.text) for each in root[i][5]]
        face = img[info[1]:info[3], info[0]:info[2]]
        face = cv2.resize(face, required_size)
        face = img_to_array(face)
        face = preprocess_input(face)
        #display('first',face)
        imgs.append(face)
    return imgs, labels

def load_extra_data(directory):
  print("[INFO] loading extra images...")
  imagePaths = list(paths.list_images(directory))
  faces = []
  labels = []

  # loop over the image paths
  for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    face = cv2.imread(imagePath)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # load the input image (224x224) and preprocess it
    face = detectFace(face)
    #display(face.shape)
    if face.shape[0] > 0 and face.shape[1] > 0 and face.shape[2] > 0:
      face = cv2.resize(face, (224, 224))
      #cv2_imshow(face)
      face = img_to_array(face)
      face = preprocess_input(face)
      # update the data and labels lists, respectively
      faces.append(face)
     #display(label,imagePath)
      labels.append(label)
    #break
  return faces, labels

# load images and extract faces for all images in a directory
def load_dataset(directory):
    x = []
    y = []
    # enumerate files
    for file in os.listdir(directory+"annotations/"):
        # get face
        faces, labels = extract_faces_labels(directory+"annotations/" + file)
        # store
        x.extend(faces)
        y.extend(labels)
        #break
    extra_faces, extra_labels = load_extra_data(directory+"extra/")
    x.extend(extra_faces)
    y.extend(extra_labels)
    return np.asarray(x), np.asarray(y)

#extra_faces, extra_labels = load_extra_data(direc+"extra/")

[INFO] loading face detector model...


In [ ]:
data, labels = load_dataset(direc)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)
# labels

In [5]:
#len(labels)
#extra_labels = np.array(extra_labels)
#data.shape
#data
labels[2]

'with_mask'

In [ ]:
# Perform LabelEncoder encoding on the labels
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# We’ll be applying on-the-fly mutations to our images in an effort to improve generalization. 
# This is known as data augmentation

aug = ImageDataGenerator(
    zoom_range=0.05,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
    )

## Prepare MobileNetV2 for fine-tuning

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.6)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# partition the data into training and testing splits using 70% of
# the data for training and the remaining 30% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.3, stratify=labels, random_state=42)

## Compiling the model

With our data prepared and model architecture in place for fine-tuning, we’re now ready to compile and train our face mask detector network:

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 50
BS = 1

In [ ]:
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

## Evaluating the model

Once training is complete, we’ll evaluate the resulting model on the test set

In [ ]:
# make predictions on the testing set

print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save(direc+'mask_detector3.model', save_format="h5")
#model_index = model_index + 1

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.show()

## USAGE

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
import os

direc = '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/'
face = direc+"face_detector"
modelname = direc+"mask_detector3.model"

classes = ["mask_weared_incorrect","with_mask", "without_mask"]
Colors = [ (255, 0, 0), (0, 255, 0), (0, 0, 255)]

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
modelFile = os.path.sep.join([face, "res10_300x300_ssd_iter_140000_fp16.caffemodel"])
configFile = os.path.sep.join([face, "deploy.prototxt"])
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model = load_model(modelname)


In [ ]:
def detectFaceOpenCVDnn(net, frame):
    frameOpencvDnn = frame.copy()
    (frameHeight,frameWidth) = frameOpencvDnn.shape[:2]
    conf_threshold = 0.5
    # construct a blob from the image
    # applying mean subtraction of values (104, 177, 123) for each blue, green and red channels correspondingly.
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], False, False)
    
    # pass the blob through the network and obtain the face detections
    #print("[INFO] computing face detections...")
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
		        # display the label and bounding box rectangle on the output
		        # frame
            # extract the face ROI, convert it from BGR to RGB channel
		        # ordering, resize it to 224x224, and preprocess it
            face = frame[y1:y2, x1:x2]
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
		        # pass the face through the model to determine if the face
		        # has a mask or not
            pred = model.predict(face)
		        # determine the class label and color we'll use to draw
		        # the bounding box and text
            predIdxs = np.argmax(pred, axis=1)
            label = classes[predIdxs[0]]
            color = Colors[predIdxs[0]]
            #display(pred,predIdxs,predIdxs[0])
		        # include the probability in the label
            percentage = "{:.2f}%".format(max(pred[0]) * 100)
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), color, int(round(frameHeight/150)), 8)
            cv2.putText(frameOpencvDnn, label, (x1 - 8, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.40, color, 1, cv2.LINE_AA)
            cv2.putText(frameOpencvDnn, percentage, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.40, color, 1, cv2.LINE_AA)
    
    # show the output image
    cv2_imshow(frameOpencvDnn)        

###USAGE on Image

In [ ]:
# load the input image from disk, clone it, and grab the image spatial
# dimensions
#imagename = direc+"Examples/example_01.png"
imagename = '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/images/maksssksksss387.png'
image = cv2.imread(imagename)
detectFaceOpenCVDnn(net,image)

###Usage for Video Stream

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=1.0):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      // show the video in the HTML element
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // prints the logs to cell
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      for (let i = 0; i < 5; i++) {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        jsLog(i + "sending")
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_algo', [img], {});
        jsLog(i + "SENT")

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));
      }

      stream.getVideoTracks()[0].stop(); // stop video stream
    }
    ''')
  display(js) # make the provided HTML, part of the cell
  data = eval_js('takePhoto({})'.format(quality)) # call the takePhoto() JavaScript function

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import dlib
from google.colab.patches import cv2_imshow
import cv2
import os

direc = '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/'
face = direc+"face_detector"
modelname = direc+"mask_detector2.model"
conf_threshold = 0.5
classes = ["mask_weared_incorrect","with_mask", "without_mask"]
Colors = [ (255, 0, 0), (0, 255, 0), (0, 0, 255)]

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
modelFile = os.path.sep.join([face, "res10_300x300_ssd_iter_140000_fp16.caffemodel"])
configFile = os.path.sep.join([face, "deploy.prototxt"])
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model = load_model(modelname)


In [ ]:
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow
import time
import sys
import numpy as np
import cv2
from PIL import Image
from io import BytesIO
import base64
import logging

def data_uri_to_img(uri):
  """convert base64image to numpy array"""
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = Image.open(BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8); 
    return image
  except Exception as e:
    logging.exception(e);print('\n')
    return None

def run_algo(imgB64):
  """
  in Colab, run_algo function gets invoked by the JavaScript, that sends N images every second

  params:
    image: image
  """
  image = data_uri_to_img(imgB64)  
  if image is None:
    print("At run_algo(): image is None.")
    return

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  try:
    # Run detection
    detectFaceOpenCVDnn(net,image)
  except Exception as e:
    logging.exception(e)
    print('\n'+e)
# register this function, so JS code could call this
output.register_callback('notebook.run_algo', run_algo)

# put the JS code in cell and run it
take_photo()

## MTCNN FACE DETECTION

In [ ]:
!pip install mtcnn

In [ ]:
# face detection with mtcnn on a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN

# draw an image with detected objects
def draw_image_with_boxes(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot the image
	pyplot.imshow(data)
	# get the context for drawing boxes
	ax = pyplot.gca()
	# plot each box
	for result in result_list:
		# get coordinates
		x, y, width, height = result['box']
		# create the shape
		rect = Rectangle((x, y), width, height, fill=False, color='red')
		# draw the box
		ax.add_patch(rect)
		# draw the dots
		for key, value in result['keypoints'].items():
			# create and draw dot
			dot = Circle(value, radius=2, color='red')
			ax.add_patch(dot)
	# show the plot
	pyplot.show()

filename = '/content/example_03.png'
# load image from file
pixels = cv2.imread(filename)
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
faces = detector.detect_faces(pixels)
# display faces on the original image
draw_image_with_boxes(filename, faces)

In [ ]:
#@title Download face detector model

import requests
CHUNK_SIZE = 32768
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params={'id': id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)



def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

    
def save_response_content(response, destination):
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


download_file_from_google_drive('1is7Ldv9ASYNcrv2GyXS7EaV58UaqhuFQ', direc+'rfcn_resnet101_widerface_91674.tar.gz')

In [ ]:
#@title Unzip Model
 import tarfile
 def extract_tar_file(zip_file_name, destination):
        zip_ref = tarfile.TarFile.open(zip_file_name, 'r')
        zip_ref.extractall(destination)
        zip_ref.close()

extract_tar_file(direc+'rfcn_resnet101_widerface_91674.tar.gz',direc+'rfcn_resnet101_widerface_91674')

In [ ]:
#@title Remove Folders or Files
!rm -r '/content/drive/My Drive/Colab Notebooks/face-masks-dataset/rfcn_resnet101_widerface_91674/'